# Computer Equipment Recommendation System

This notebook demonstrates how to build a machine learning model that predicts and suggests computer equipment for a computer shop based on user input and current trends.

In [17]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [18]:
# Load and Preprocess Computer Equipment Dataset
data = pd.read_csv("Computer_Equipment_Dataset.csv")
# Fill missing values if any
data.fillna("Unknown", inplace=True)
data.head()

,Name,Category,Description,Price,Brand,Trend
0,Laptop,Computers,High-performance laptop for gaming and work,1200,Dell,Trending
1,Mechanical Keyboard,Accessories,RGB backlit mechanical keyboard,100,Logitech,Popular
2,Gaming Mouse,Accessories,High DPI gaming mouse,60,Razer,Trending
3,Monitor,Displays,27-inch 4K UHD monitor,350,Samsung,Trending
4,SSD,Storage,1TB NVMe SSD for fast storage,90,WD,Popular


In [19]:
# Feature Engineering for Equipment Suggestions
# Combine text features for TF-IDF
text_features = data['Name'] + " " + data['Category'] + " " + data['Description'] + " " + data['Brand']

# Numerical features: Price
scaler = StandardScaler()
price_scaled = scaler.fit_transform(data[['Price']])

In [20]:
# Train Machine Learning Model for Equipment Prediction
# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(text_features)

# Combine TF-IDF and scaled price features
from scipy.sparse import hstack
features = hstack([tfidf_matrix, price_scaled])

# Train KNN model
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(features)

,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [21]:
# Evaluate Model Performance
# Check average distance to nearest neighbors
neighbors_distances, _ = knn.kneighbors(features)
avg_distance = np.mean(neighbors_distances)
print(f"Average distance to nearest neighbors: {avg_distance:.4f}")

Average distance to nearest neighbors: 0.6384


In [22]:
# Implement Suggestion Functionality
def suggest_equipment(equipment_name, top_n=5):
    idx = data[data['Name'].str.lower() == equipment_name.lower()].index
    if len(idx) == 0:
        print("Equipment not found.")
        return []
    idx = idx[0]
    # Use getrow to extract a single row as 2D matrix for kneighbors
    distances, indices = knn.kneighbors(features.getrow(idx), n_neighbors=top_n+1)
    suggestions = data.iloc[indices[0][1:]]  # Exclude the selected item itself
    return suggestions[['Name', 'Category', 'Brand', 'Price', 'Trend']]

# Example usage:
suggest_equipment('Laptop')

,Name,Category,Brand,Price,Trend
5,Graphics Card,Components,Nvidia,700,Trending
3,Monitor,Displays,Samsung,350,Trending
7,Printer,Peripherals,HP,150,Popular
6,Router,Networking,TP-Link,130,Popular
9,Headset,Accessories,HyperX,120,Trending


In [23]:
# Display Current Trends in Computer Equipment
trending_items = data[data['Trend'] == 'Trending']
print("Trending Computer Equipment:")
print(trending_items[['Name', 'Category', 'Brand', 'Price']])

Trending Computer Equipment:
            Name     Category     Brand  Price
0         Laptop    Computers      Dell   1200
2   Gaming Mouse  Accessories     Razer     60
3        Monitor     Displays   Samsung    350
5  Graphics Card   Components    Nvidia    700
8         Webcam  Accessories  Logitech     80
9        Headset  Accessories    HyperX    120


In [24]:
# Load and Combine All Equipment Datasets
import pandas as pd
import numpy as np

# Load GPU dataset
gpu_df = pd.read_csv('datasets/All_GPUs.csv')
# Load CPU dataset
cpu_df = pd.read_csv('datasets/CPU.csv')
# Load other components dataset
other_df = pd.read_csv('datasets/datasets.csv')

# Preprocess GPU dataset
gpu_df['Type'] = 'GPU'
gpu_df['Name'] = gpu_df['Name'].fillna(gpu_df['Manufacturer'] + ' ' + gpu_df['Architecture'].astype(str))
gpu_df['Description'] = gpu_df['Architecture'].astype(str) + ' ' + gpu_df['Memory_Type'].astype(str) + ' ' + gpu_df['Memory'].astype(str)
gpu_df['Brand'] = gpu_df['Manufacturer']
gpu_df['Price'] = gpu_df['Release_Price'].replace('[\$,]', '', regex=True).astype(str).str.extract('([\d.]+)').fillna('0').astype(float)

# Preprocess CPU dataset
cpu_df['Type'] = 'CPU'
cpu_df['Name'] = cpu_df['name']
cpu_df['Description'] = cpu_df['brand'] + ' ' + cpu_df['socket'] + ' ' + cpu_df['speed'].astype(str) + 'GHz'
cpu_df['Brand'] = cpu_df['brand']
cpu_df['Price'] = cpu_df['power'].astype(float) * 2  # Example: estimate price from power

# Preprocess other components dataset
other_df['Type'] = 'Other'
other_df['Name'] = other_df['CLASS']
other_df['Description'] = other_df['CLASS']
other_df['Brand'] = 'Unknown'
other_df['Price'] = other_df['TRAIN'].astype(float) / 10  # Example: estimate price from TRAIN

# Select common columns
gpu_cols = ['Type', 'Name', 'Description', 'Brand', 'Price']
cpu_cols = ['Type', 'Name', 'Description', 'Brand', 'Price']
other_cols = ['Type', 'Name', 'Description', 'Brand', 'Price']

# Combine all
all_data = pd.concat([
    gpu_df[gpu_cols],
    cpu_df[cpu_cols],
    other_df[other_cols]
], ignore_index=True)

# Fill missing values if any
all_data.fillna('Unknown', inplace=True)
all_data.head()

,Type,Name,Description,Brand,Price
0,GPU,GeForce GTS 150,Tesla G92b GDDR3 1024 MB,Nvidia,0.0
1,GPU,Radeon HD 2900 XT 512MB,R600 XT GDDR3 512 MB,AMD,0.0
2,GPU,Radeon HD 2900 Pro,R600 PRO GDDR3 512 MB,AMD,0.0
3,GPU,Radeon HD 2600 XT Diamond Edition,RV630 GDDR4 256 MB,AMD,0.0
4,GPU,Radeon HD 2600 XT,RV630 GDDR3 256 MB,AMD,0.0


In [25]:
# Save the Model and Preprocessing Objects
import pickle

with open('knn_model_equipment.pickle', 'wb') as f:
    pickle.dump(knn, f)
with open('tfidf_equipment.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)
with open('scaler_equipment.pickle', 'wb') as f:
    pickle.dump(scaler, f)

# Save the processed dataset
all_data.to_pickle('Computer_Equipment_Recommend_Dataset.pickle')